In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import geopandas as gpd
import requests
from requests import Request
from owslib.wfs import WebFeatureService
from owslib.etree import etree
from owslib.fes import *

ppdm_url = 'http://explorer-portal.australiaeast.cloudapp.azure.com/geoserver/mbie/ows'

filter = PropertyIsEqualTo(propertyname='FIELD', literal='Waihapa')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")

params = dict(
    service='WFS', 
    version="2.0.0",
    request='GetFeature',
    typeName='mbie:ProductionData',
    outputFormat='application/json',
    filter=filterxml
)
q = Request('GET', ppdm_url, params=params).prepare().url

r = requests.get(url=q)

ppdm_df = gpd.read_file(q)

ppdm_df.head()

,id,FIELD,POOL,UWI,WELLNAME,MATERIAL,DATE,OILPRODM3,DGASPROKSM,AGASPROKSM,GCONDVOL,WATPRODVOL,geometry
0,ProductionData.fid--6754a5bd_16dd1f2ef31_-1e7b,Waihapa,Waihapa,100000166117,Ngaere-2,OIL,8/09/2015,28073,None,62049,None,25122,None
1,ProductionData.fid--6754a5bd_16dd1f2ef31_-1e7a,Waihapa,Waihapa,100000166117,Ngaere-2,OIL,9/09/2015,31914,None,79030,None,61272,None
2,ProductionData.fid--6754a5bd_16dd1f2ef31_-1e79,Waihapa,Waihapa,100000166117,Ngaere-2,OIL,10/09/2015,24728,None,75399,None,24316,None
3,ProductionData.fid--6754a5bd_16dd1f2ef31_-1e78,Waihapa,Waihapa,100000166117,Ngaere-2,OIL,11/09/2015,27873,None,77873,None,41318,None
4,ProductionData.fid--6754a5bd_16dd1f2ef31_-1e77,Waihapa,Waihapa,100000166117,Ngaere-2,OIL,12/09/2015,28933,None,73728,None,54927,None


In [44]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(x=ppdm_df.WELLNAME, y=ppdm_df.AGASPROKSM, name='AGASPROKSM'))
fig.add_trace(go.Bar(x=ppdm_df.WELLNAME, y=ppdm_df.DGASPROKSM, name='DGASPROKSM'))
fig.add_trace(go.Bar(x=ppdm_df.WELLNAME, y=ppdm_df.GCONDVOL, name='GCONDVOL'))
fig.add_trace(go.Bar(x=ppdm_df.WELLNAME, y=ppdm_df.WATPRODVOL, name='WATPRODVOL'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [61]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

params = dict(
    service='WFS', 
    version="2.0.0", 
    request='GetFeature', 
    typeName='EXTERNAL_Petroleum_Data:Petroleum_fields', 
    outputFormat='geojson',
    srsname='urn:ogc:def:crs:EPSG::4326',
    filter='<Filter><PropertyIsEqualTo><PropertyName>NAME</PropertyName><Literal>Waihapa-Ngaere</Literal></PropertyIsEqualTo></Filter>',
)
q = Request('GET', 'http://data.nzpam.govt.nz/arcgis/services/EXTERNAL/Petroleum_Data/MapServer/WFSServer', params=params).prepare().url

print (q)

petro_fields_df = gpd.read_file(q)

petro_fields_df.head()

http://data.nzpam.govt.nz/arcgis/services/EXTERNAL/Petroleum_Data/MapServer/WFSServer?service=WFS&version=2.0.0&request=GetFeature&typeName=EXTERNAL_Petroleum_Data%3APetroleum_fields&outputFormat=geojson&srsname=urn%3Aogc%3Adef%3Acrs%3AEPSG%3A%3A4326&filter=%3CFilter%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3ENAME%3C%2FPropertyName%3E%3CLiteral%3EWaihapa-Ngaere%3C%2FLiteral%3E%3C%2FPropertyIsEqualTo%3E%3C%2FFilter%3E


,OBJECTID,NAME,COMMODITY,OPERATION_NAME,OPERATOR,PRODUCING_FORMATION,AGE,Shape_Length,Shape_Area,geometry
0,75,Waihapa-Ngaere,Oil,TAWN,NZEC,Tikorangi FM,Oligocene,22195.462383,1.749802e+07,POLYGON ((174.3745965477475 -39.34648962919061...


In [ ]:
import pandas as pd
import requests
import io
import plotly.express as px
import os

url = 'https://www.mbie.govt.nz/assets/Data-Files/Energy/nz-energy-quarterly-and-energy-in-nz/Oil.xlsx'

df = pd.read_excel(url, 'Annual_mmbls')
df = df.T

new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df = df.dropna(axis=1, how='all')

if not os.path.exists("images"):
    os.mkdir("images")

fig = px.line(df, x=" Calendar year", y="Waihapa", title='Waihapa Petroleum Field')
fig.layout.yaxis.title = 'Supply in Waihapa (mmbls)'
fig.write_image("images/waihapa.png")

fig.show()

In [74]:
from ipyleaflet import Map, GeoData, LayersControl, WidgetControl
from ipywidgets import Text, HTML, Layout
import textwrap

def click_handler (feature=None, **kwargs):
    if feature is not None:        
        html1.value = ''
        items = feature['properties']
        for item in items:
            html1.value += '<h6>{}: <b>{}</b></h6>'.format(item, textwrap.shorten(str(items[item]), width=40))

geo_data = GeoData(
    geo_dataframe=petro_fields_df,
    style={'color': 'red', 'fillColor': '#cc3333', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'blue' , 'fillOpacity': 0.2},
    name='Waihapa Ngaere - Petroleum Field'
)
geo_data.on_click(click_handler)

m = Map(center=(-39.4305, 174.2993), zoom=10, layout=dict(height='800px'))
m.add_layer(geo_data)

image = Image(value='', layout=Layout(height='100px', width='100px'))
image.layout.margin = '0px 10px 10px 10px'
control = WidgetControl(widget=html1, position='bottomright')
m.add_control(control)
m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [83]:
import pandas as pd
import requests
import io
import plotly.express as px
import os

url = 'https://www.mbie.govt.nz/assets/Data-Files/Energy/nz-energy-quarterly-and-energy-in-nz/Oil.xlsx'

df = pd.read_excel(url, 'Annual_mmbls')
df = df.T

new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df = df.dropna(axis=1, how='all')

if not os.path.exists("images"):
    os.mkdir("images")

fig = px.line(df, x=" Calendar year", y="Waihapa", title='Waihapa Petroleum Field')
fig.layout.yaxis.title = 'Supply in Waihapa (mmbls)'

fig.show()